# Best bus stops

Find the best bus stops from PTV data.


Many bus routes don't have night services. So I'd like to find the bus stops that have night services.

In [1]:
import components as comp
import pandas as pd
import numpy as np

In [2]:
# Utilize a custom module to convert PTV's gtfs.zip data into a Series of Pandas DataFrames

df = comp.process_gtfs_zip('http://data.ptv.vic.gov.au/downloads/gtfs.zip', '')
# 2m30s

In [3]:
ptv = df.set_index(['branch_id', 'table_name'], inplace=False)['df']

In [4]:
# Create a DataFrame for each bus table in the GTFS data
df_bus_agency : pd.DataFrame = ptv['4']['agency']
df_bus_calendar : pd.DataFrame = ptv['4']['calendar']
df_bus_calendar_dates : pd.DataFrame = ptv['4']['calendar_dates']
df_bus_routes : pd.DataFrame = ptv['4']['routes']
df_bus_shapes : pd.DataFrame = ptv['4']['shapes']
df_bus_stops : pd.DataFrame = ptv['4']['stops']
df_bus_stop_times : pd.DataFrame = ptv['4']['stop_times']
df_bus_trips : pd.DataFrame = ptv['4']['trips']

In [5]:
def get_max_row(df : pd.DataFrame, by: str):
    """
    Return the row with the maximum/minimum value of 'by' column in the given dataframe

    df.max() and df.min() only returns the considered column instead of the whole row.
    """
    return df.loc[df[by].idxmax()]

def get_min_row(df : pd.DataFrame, by: str):
    """
    Return the row with the maximum/minimum value of 'by' column in the given dataframe

    df.max() and df.min() only returns the considered column instead of the whole row.
    """
    return df.loc[df[by].idxmin()]

In [6]:
days_in_week = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']

In [7]:
dfbst = pd.merge(df_bus_stop_times, df_bus_trips, on='trip_id', how='left')
dfbst = pd.merge(dfbst, df_bus_calendar, on='service_id', how='left')
dfbst = dfbst[['stop_id', 'trip_id', 'route_id', 'arrival_time', 'start_date', 'end_date'] + days_in_week]

# 10s

In [8]:
dk_sr = {}
for d in days_in_week:
    dk_sr[d] = dfbst[dfbst[d] == 1].groupby(['stop_id', 'route_id'])['arrival_time'].max()
    dk_sr[d].rename(d, inplace=True)

# 30s


In [9]:
dfb_sr = pd.merge(dk_sr['monday'], dk_sr['tuesday'], on=['stop_id', 'route_id'], how='outer')
for d in days_in_week[2:]:
    dfb_sr = pd.merge(dfb_sr, dk_sr[d], on=['stop_id', 'route_id'], how='outer')
dfb_sr.dropna(inplace=True)
dfb_sr['weekday'] = dfb_sr[['monday', 'tuesday', 'wednesday', 'thursday', 'friday']].min(axis=1)
dfb_sr['weekend'] = dfb_sr[['saturday', 'sunday']].min(axis=1)
dfb_sr['week'] = dfb_sr.min(axis=1)
dfb_sr.sort_values(by='week', inplace=True, ascending=False)
dfb_sr.reset_index(inplace=True)

In [10]:
dfb_sr.head(5)

,stop_id,route_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,weekday,weekend,week
0,21048,14-907-aus-1,24:52:00,24:52:00,24:52:00,24:52:00,26:42:00,27:37:00,24:43:00,24:52:00,24:43:00,24:43:00
1,47684,14-907-aus-1,24:51:00,24:51:00,24:51:00,24:51:00,26:41:00,27:36:00,24:42:00,24:51:00,24:42:00,24:42:00
2,44914,14-908-aus-1,24:50:00,24:50:00,24:50:00,24:50:00,26:40:00,27:35:00,24:42:00,24:50:00,24:42:00,24:42:00
3,3965,14-907-aus-1,24:50:00,24:50:00,24:50:00,24:50:00,26:40:00,27:35:00,24:41:00,24:50:00,24:41:00,24:41:00
4,3966,14-907-aus-1,24:50:00,24:50:00,24:50:00,24:50:00,26:40:00,27:35:00,24:41:00,24:50:00,24:41:00,24:41:00


In [11]:
dfb_sr_info = pd.merge(dfb_sr, df_bus_stops, on='stop_id', how='left')
dfb_sr_info = pd.merge(dfb_sr_info, df_bus_routes, on='route_id', how='left')

In [12]:
dfb_sr_info.head(5)

,stop_id,route_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,weekday,...,week,stop_name,stop_lat,stop_lon,agency_id,route_short_name,route_long_name,route_type,route_color,route_text_color
0,21048,14-907-aus-1,24:52:00,24:52:00,24:52:00,24:52:00,26:42:00,27:37:00,24:43:00,24:52:00,...,24:43:00,Mitcham Station/Colombo St (Mitcham),-37.818058,145.192188,NaN,907,Mitcham - City (King/Lonsdale Sts),3,FF8200,FFFFFF
1,47684,14-907-aus-1,24:51:00,24:51:00,24:51:00,24:51:00,26:41:00,27:36:00,24:42:00,24:51:00,...,24:42:00,Harrison St/Mitcham Rd (Mitcham),-37.813988,145.193560,NaN,907,Mitcham - City (King/Lonsdale Sts),3,FF8200,FFFFFF
2,44914,14-908-aus-1,24:50:00,24:50:00,24:50:00,24:50:00,26:40:00,27:35:00,24:42:00,24:50:00,...,24:42:00,The Pines SC/Reynolds Rd (Doncaster East),-37.762878,145.168921,NaN,908,The Pines SC - City (King/Lonsdale Sts),3,FF8200,FFFFFF
3,3965,14-907-aus-1,24:50:00,24:50:00,24:50:00,24:50:00,26:40:00,27:35:00,24:41:00,24:50:00,...,24:41:00,Burnett St/Mitcham Rd (Mitcham),-37.812065,145.193368,NaN,907,Mitcham - City (King/Lonsdale Sts),3,FF8200,FFFFFF
4,3966,14-907-aus-1,24:50:00,24:50:00,24:50:00,24:50:00,26:40:00,27:35:00,24:41:00,24:50:00,...,24:41:00,Doncaster East Rd/Mitcham Rd (Mitcham),-37.809411,145.193081,NaN,907,Mitcham - City (King/Lonsdale Sts),3,FF8200,FFFFFF


In [13]:
dk_s = {}
for d in days_in_week:
    dk_s[d] = dfbst[dfbst[d] == 1].groupby('stop_id')['arrival_time'].max()
    dk_s[d].rename(d, inplace=True)

In [14]:
dfb_s = pd.merge(dk_s['monday'], dk_s['tuesday'], on='stop_id', how='outer')
for d in days_in_week[2:]:
    dfb_s = pd.merge(dfb_s, dk_s[d], on='stop_id', how='outer')
dfb_s.dropna(inplace=True)
dfb_s['weekday'] = dfb_s[['monday', 'tuesday', 'wednesday', 'thursday', 'friday']].min(axis=1)
dfb_s['weekend'] = dfb_s[['saturday', 'sunday']].min(axis=1)
dfb_s['week'] = dfb_s.min(axis=1)
dfb_s.sort_values(by='week', inplace=True, ascending=False)
dfb_s.reset_index(inplace=True)

In [15]:
dfb_s_info = pd.merge(dfb_s, df_bus_stops, on='stop_id', how='left')

In [16]:
dk_r = {}
for d in days_in_week:
    dk_r[d] = dfbst[dfbst[d] == 1].groupby('route_id')['arrival_time'].max()
    dk_r[d].rename(d, inplace=True)

In [17]:
dfb_r = pd.merge(dk_r['monday'], dk_r['tuesday'], on='route_id', how='outer')
for d in days_in_week[2:]:
    dfb_r = pd.merge(dfb_r, dk_r[d], on='route_id', how='outer')
dfb_r.dropna(inplace=True)
dfb_r['weekday'] = dfb_r[['monday', 'tuesday', 'wednesday', 'thursday', 'friday']].min(axis=1)
dfb_r['weekend'] = dfb_r[['saturday', 'sunday']].min(axis=1)
dfb_r['week'] = dfb_r.min(axis=1)
dfb_r.sort_values(by='week', inplace=True, ascending=False)
dfb_r.reset_index(inplace=True)

In [18]:
dfb_r_info = pd.merge(dfb_r, df_bus_routes, on='route_id', how='left')

In [19]:
dfb_r_info.head(5)

,route_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,weekday,weekend,week,agency_id,route_short_name,route_long_name,route_type,route_color,route_text_color
0,14-907-aus-1,24:52:00,24:52:00,24:52:00,24:52:00,26:42:00,27:45:00,24:43:00,24:52:00,24:43:00,24:43:00,NaN,907,Mitcham - City (King/Lonsdale Sts),3,FF8200,FFFFFF
1,14-908-aus-1,24:50:00,24:50:00,24:50:00,24:50:00,26:40:00,27:35:00,24:42:00,24:50:00,24:42:00,24:42:00,NaN,908,The Pines SC - City (King/Lonsdale Sts),3,FF8200,FFFFFF
2,14-905-aus-1,24:32:00,24:32:00,24:32:00,24:32:00,26:20:00,27:15:00,24:37:00,24:32:00,24:37:00,24:32:00,NaN,905,The Pines SC - City (King/Lonsdale Sts),3,FF8200,FFFFFF
3,14-906-aus-1,24:44:00,24:44:00,24:44:00,24:44:00,24:44:00,24:30:00,24:29:00,24:44:00,24:29:00,24:29:00,NaN,906,Warrandyte - City (King/Lonsdale Sts),3,FF8200,FFFFFF
4,35-426-aus-1,24:51:00,24:51:00,24:51:00,24:51:00,24:51:00,25:22:00,24:20:00,24:51:00,24:20:00,24:20:00,NaN,426,Caroline Springs - Sunshine Station,3,FF8200,FFFFFF


In [20]:
dfb_s_info.head(5)

,stop_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,weekday,weekend,week,stop_name,stop_lat,stop_lon
0,21048,24:52:00,24:52:00,24:52:00,24:52:00,26:42:00,27:37:00,24:43:00,24:52:00,24:43:00,24:43:00,Mitcham Station/Colombo St (Mitcham),-37.818058,145.192188
1,47684,24:51:00,24:51:00,24:51:00,24:51:00,26:41:00,27:36:00,24:42:00,24:51:00,24:42:00,24:42:00,Harrison St/Mitcham Rd (Mitcham),-37.813988,145.193560
2,44914,24:50:00,24:50:00,24:50:00,24:50:00,26:40:00,27:35:00,24:42:00,24:50:00,24:42:00,24:42:00,The Pines SC/Reynolds Rd (Doncaster East),-37.762878,145.168921
3,3966,24:50:00,24:50:00,24:50:00,24:50:00,26:40:00,27:35:00,24:41:00,24:50:00,24:41:00,24:41:00,Doncaster East Rd/Mitcham Rd (Mitcham),-37.809411,145.193081
4,3965,24:50:00,24:50:00,24:50:00,24:50:00,26:40:00,27:35:00,24:41:00,24:50:00,24:41:00,24:41:00,Burnett St/Mitcham Rd (Mitcham),-37.812065,145.193368
